<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/RandomPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#@title Upgrade to TensorFlow nightly
!pip install --upgrade tf-nightly

     |████████████████████████████████| 398.7MB 45kB/s 
     |████████████████████████████████| 1.3MB 41.9MB/s 
     |████████████████████████████████| 11.9MB 25.1MB/s 
     |████████████████████████████████| 3.8MB 33.8MB/s 
     |████████████████████████████████| 14.5MB 149kB/s 
     |████████████████████████████████| 471kB 41.4MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.1

In [1]:
import tensorflow as tf

In [2]:
from scipy.special import erf
import numpy as np
import time

In [3]:
x = tf.math.sobol_sample(10,10)

In [4]:
import numpy as np

In [5]:
class BrownianBridge():
    def __init__(self, number_time_steps):
        super(BrownianBridge, self).__init__()

        self.number_time_steps = number_time_steps

        self.left_index = np.zeros(number_time_steps, dtype=int)
        self.right_index = np.zeros(number_time_steps, dtype=int)
        self.bridge_index = np.zeros(number_time_steps, dtype=int)
        self.left_weight = np.zeros(number_time_steps)
        self.right_weight = np.zeros(number_time_steps)
        self.std_dev = np.zeros(number_time_steps)

        self._map = np.zeros(number_time_steps, dtype=int)

        self._map[-1] = 1
        self.bridge_index[0] = number_time_steps - 1
        self.std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
        self.left_weight[0] = 0
        self.right_weight[0] = 0

        j=0
        for i in range(1,number_time_steps):
            while self._map[j] == True:
                j = j + 1
            k = j
            while self._map[k] == False:
                k = k + 1
            l = j+((k-1-j)>>1)
            self._map[l]=i
            self.bridge_index[i]=l
            self.left_index[i]=j
            self.right_index[i]=k
            self.left_weight[i]=(k-l)/(k+1-j)
            self.right_weight[i]=(1+l-j)/(k+1-j)
            self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
            j=k+1
            if j>=number_time_steps:
                j=0

    @tf.function(jit_compile=True)
    def buildPath(self, z, number):
        path = tf.unstack(tf.zeros(shape=(self.number_time_steps,number)))
        path[-1] = self.std_dev[0]*z[0];
        for i in np.arange(1,self.number_time_steps):
            j = self.left_index[i]
            k = self.right_index[i]
            l = self.bridge_index[i]
            #if j > 0:
            #    path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
            #else:
            path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]

        return path

In [6]:
import tensorflow as tf

In [327]:
import numpy as np
import math

In [328]:
number_time_steps = 256
number = 3

left_index = np.zeros(number_time_steps, dtype=int)
right_index = np.zeros(number_time_steps, dtype=int)
bridge_index = np.zeros(number_time_steps, dtype=int)
left_weight = np.zeros(number_time_steps)
right_weight = np.zeros(number_time_steps)
std_dev = np.zeros(number_time_steps)

_map = np.zeros(number_time_steps, dtype=int)

_map[-1] = 1
bridge_index[0] = number_time_steps - 1
std_dev[0] = math.sqrt(1.0 * number_time_steps)
left_weight[0] = 0
right_weight[0] = 0

j=0
for i in range(1,number_time_steps):
  while _map[j] == True:
    j = j + 1
  k = j
  while _map[k] == False:
    k = k + 1
l = j+((k-1-j)>>1)
_map[l]=i
bridge_index[i]=l
left_index[i]=j
right_index[i]=k
left_weight[i]=(k-l)/(k+1-j)
right_weight[i]=(1+l-j)/(k+1-j)
std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
j=k+1
if j>=number_time_steps:
  j=0

In [333]:
bridge_index_t = tf.constant(bridge_index)
right_index_t = tf.constant(right_index)
left_index_t = tf.constant(left_index)
left_weight_t = tf.constant(left_weight)
right_weight_t = tf.constant(right_weight)
std_dev_t = tf.constant(std_dev)

print(left_index_t)

tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(256,), dtype=int64)


In [401]:
#@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None), dtype=tf.float64), tf.TensorSpec(shape=(), dtype=tf.int32)))
@tf.function(jit_compile=True)
def test(z, number):
  print("Tracing with", number)

  path = tf.unstack(tf.zeros(shape=(8,3),dtype=tf.dtypes.float64))
  path[-1] = z[0]
  print(path)
  print(z)
  for i in range(number-1):
    print(i)  
    print(z[i])
    #print(path[i])
    #tmp = z[i]
    path[i] = z[i]

  return path

In [404]:
number = tf.constant(3)

x = tf.zeros(shape=(3,1),dtype=tf.dtypes.float64)

test(x,number)

print(x)

Tracing with Tensor("number:0", shape=(), dtype=int32)
[<tf.Tensor 'unstack:0' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:1' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:2' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:3' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:4' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:5' shape=(3,) dtype=float64>, <tf.Tensor 'unstack:6' shape=(3,) dtype=float64>, <tf.Tensor 'strided_slice:0' shape=(1,) dtype=float64>]
Tensor("z:0", shape=(3, 1), dtype=float64)
Tensor("while/Placeholder:0", shape=(), dtype=int32)
Tensor("while/strided_slice:0", shape=(1,), dtype=float64)


TypeError: ignored

In [ ]:
print(tf.autograph.to_code(test.python_function))

In [403]:
print(test.pretty_printed_concrete_signatures())

test(z, number=3)
  Args:
    z: float64 Tensor, shape=(3, 1)
  Returns:
    [<1>, <2>, <3>, <4>, <5>, <6>, <7>, <8>]
      <1>: float64 Tensor, shape=(1,)
      <2>: float64 Tensor, shape=(1,)
      <3>: float64 Tensor, shape=(3,)
      <4>: float64 Tensor, shape=(3,)
      <5>: float64 Tensor, shape=(3,)
      <6>: float64 Tensor, shape=(3,)
      <7>: float64 Tensor, shape=(3,)
      <8>: float64 Tensor, shape=(1,)


In [ ]:
#@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64), 
#                                                tf.TensorSpec(shape=(), dtype=tf.int32),
#                                                tf.TensorSpec(shape=(), dtype=tf.int32),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.int64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
#                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
@tf.function(jit_compile=True)
def buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
  print("Tracing with", number)
  path = tf.unstack(tf.zeros(shape=(number_time_steps, number),dtype=tf.dtypes.float64))
  path[-1] = std_dev[0]*z[0];
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
#    x = z[i]
#    print("x")
#    print(x)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    if j > 0:
      path[l] = left_weight[i] * path[j-1] + right_weight[i] * path[k] + self.std_dev[i] * z[i]
    else:
      path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [ ]:
samples = 8
dim = 128

x = tf.math.sobol_sample(dim,samples,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

In [268]:
path = buildPath(z, samples, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

TypeError: ignored

In [ ]:
print(buildPath.pretty_printed_concrete_signatures())

In [269]:
print(tf.autograph.to_code(buildPath.python_function))

AttributeError: ignored

In [270]:
bridge = BrownianBridge(2)

In [324]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float32)
    z = tf.transpose(tf.math.erf(x))
    #w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    #w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
    #print(w[2])
time_end = time.time()

print(w)

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.40128326416015625
[<tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>, <tf.Tensor: shape=(200000,), dtype=float64, numpy=array([0., 0., 0.

In [273]:
import torch

In [274]:
@torch.jit.script
def foo(x, number: int):
  print(number)
  z = 0.
  for i in range(0,number):
    tmp = x[i]
    z = z + tmp

# See the compiled graph as Python code
print(foo.code)

def foo(x: Tensor,
    number: int) -> None:
  print(number)
  z = 0.
  for _0 in range(torch.__range_length(0, number, 1)):
    i = torch.__derive_index(_0, 0, 1)
    tmp = torch.select(x, 0, i)
    z0 = torch.add(tmp, z, 1)
    z = annotate(float, z0)
  return None



In [275]:
# Call the function using the TorchScript interpreter
x = torch.zeros(11)

foo(x, 10)

10


In [618]:
@torch.jit.script
def buildPath(path, z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  path[-1] = std_dev[0]*z[0]
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
    lw = left_weight[i]
    rw = right_weight[i]
    sd = std_dev[i]
    #print(lw)
    if j > 0:
      path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
    else:
      path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [619]:
#scripted_func = torch.jit.script(buildPath)

In [620]:
print(buildPath.code)

def buildPath(path: Tensor,
    z: Tensor,
    number: int,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> Tensor:
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    j = torch.select(left_index, 0, i)
    _4 = annotate(int, j)
    k = torch.select(right_index, 0, i)
    _5 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _6 = annotate(int, l)
    lw = torch.select(left_weight, 0, i)
    rw = torch.select(right_weight, 0, i)
    sd = torch.select(std_dev, 0, i)
    if torch.gt(_4, 0):
      _7 = torch.select(path, 0, torch.sub(_4, 1))
      _8 = torch.mul(_7, lw)
      _9 = torch.mul(torch.select(path, 0, _5), rw)
      _10 = torch.add(_8, _9, alpha=1)


In [621]:
path = torch.zeros(size=(256,2))

x = torch.randn(256,2)

left_index_t = torch.tensor(left_index)
right_index_t = torch.tensor(right_index)
bridge_index_t = torch.tensor(bridge_index)
left_weight_t = torch.tensor(left_weight)
right_weight_t = torch.tensor(right_weight)
std_dev_t = torch.tensor(std_dev)

path = buildPath(path, x, 2, 256, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

In [622]:
scripted_func.graph_for(path, x, 2, 256, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

graph(%path.1 : Tensor,
      %z.1 : Tensor,
      %number : int,
      %number_time_steps.1 : int,
      %left_index.1 : Tensor,
      %right_index.1 : Tensor,
      %bridge_index.1 : Tensor,
      %left_weight.1 : Tensor,
      %right_weight.1 : Tensor,
      %std_dev.1 : Tensor):
  %14 : int = prim::Constant[value=1]() # <ipython-input-433-652dbf0dca88>:3:8
  %j.1 : int = prim::Constant[value=0]() # <ipython-input-433-652dbf0dca88>:3:21
  %12 : int = prim::Constant[value=-1]() # <ipython-input-433-652dbf0dca88>:3:7
  %11 : bool = prim::Constant[value=0]()
  %10 : bool = prim::Constant[value=1]() # <ipython-input-433-652dbf0dca88>:8:2
  %15 : Tensor = aten::select(%std_dev.1, %j.1, %j.1) # <ipython-input-433-652dbf0dca88>:3:13
  %16 : Tensor = aten::select(%z.1, %j.1, %j.1) # <ipython-input-433-652dbf0dca88>:3:24
  %19 : Tensor = aten::mul(%15, %16) # <ipython-input-433-652dbf0dca88>:3:13
  %20 : Tensor = aten::select(%path.1, %j.1, %12) # <ipython-input-433-652dbf0dca88>:3:2
  %22 :

In [623]:
sobol_engine =  torch.quasirandom.SobolEngine(2)
sobol_engine.draw(3)

tensor([[0.5000, 0.5000],
        [0.7500, 0.2500],
        [0.2500, 0.7500]])

In [624]:
import time

In [625]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

path = torch.zeros(size=(dim,number))

sobol_engine =  torch.quasirandom.SobolEngine(dim)

# First run (includes graph optimization time)
time_start = time.time()
x = sobol_engine.draw(number)
y = torch.transpose(torch.erf(x),0,1)
#path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
x = sobol_engine.draw(number)
y = torch.transpose(torch.erf(x),0,1)
path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

# First run (includes graph optimization time)
time_start = time.time()
x = torch.randn(size=(dim,number))
#y = torch.transpose(torch.erf(x),0,1)
#path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Third time on a CPU: ", time_price_cpu)


First time on a CPU:  0.24289393424987793
Second time on a CPU:  1.1659772396087646
Third time on a CPU:  0.6965889930725098


In [615]:
import torch
import torchvision.models as models
import torch.autograd.profiler as profiler

In [628]:
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

path = torch.zeros(size=(dim,number), dtype=float)

with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        x = sobol_engine.draw(number)
        y = torch.transpose(torch.erf(x-0.5),0,1)
        path = buildPath(path, y, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)


In [629]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
              model_inference         0.44%       6.341ms       100.00%        1.446s        1.446s             1  
                    buildPath         2.87%      41.484ms        79.32%        1.147s        1.147s             1  
                    aten::mul        58.87%     851.147ms        59.26%     856.826ms       1.677ms           511  
                    aten::erf         7.08%     102.378ms        14.16%     204.781ms     102.390ms             2  
                    aten::add         7.07%     102.295ms        12.08%     174.710ms     685.137us           255  
     aten::_sobol_engine_draw         6.57%      95.002ms         8.62% 

In [469]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))


-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                   Input Shapes  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------  
              model_inference         0.39%       4.746ms       100.00%        1.204s        1.204s             1                                             []  
                    buildPath         1.93%      23.264ms        80.08%     964.157ms     964.157ms             1  [[256, 200000], [256, 200000], [], [], [256],  
                    aten::mul        65.32%     786.443ms        65.64%     790.341ms       3.099ms           255               [[], [], [200000], [], [200000]]  
     aten::_sobol_engi

In [470]:
prof.export_chrome_trace("trace.json")

In [672]:
#from numba import jit
import random

#@jit(nopython=True)
def buildPath_numpy(path, z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  path[-1] = std_dev[0]*z[0]
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
    lw = left_weight[i]
    rw = right_weight[i]
    sd = std_dev[i]
    #print(lw)
    if j > 0:
      tmp = path[j-1] * lw + path[k] * rw + z[i] * sd
    else:
      tmp = sd * z[i] #path[k] * rw + z[i,:] * sd

  return path

In [673]:
x = np.random.randn(256, number)

path_numpy = np.zeros(shape=(dim,number))

In [674]:
p = buildPath_numpy(path_numpy, x, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)

In [675]:
dim =  256#@param {type:"integer"}
number =  200000#@param {type:"integer"}

path_numpy = np.zeros(shape=(dim,number))
#path_numpy = [None] * dim#np.zeros(shape=(dim,number))


# First run (includes graph optimization time)
time_start = time.time()
x = sobol_engine.draw(number)
y = torch.erf(x).numpy().T
path = buildPath_numpy(path_numpy, y, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Third time on a CPU: ", time_price_cpu)



Third time on a CPU:  1.0028340816497803


In [676]:
%lprun buildPath_numpy(path_numpy, y, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)

In [677]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [678]:
%lprun -f buildPath_numpy buildPath_numpy(path_numpy, y, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)